# SQL Query using `pandas`

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("data_input/chinook.db")

## Database Schema

### Apa itu database schema?

Database schema atau yang disebut sebagai Entity Relationship Diagram (ERD) merupakan suatu skema yang digunakan untuk merancang suatu basis data, ada tiga komponen utama didalamnya:
1. Entitas: tabel pada sebuah database
2. Atribut: kolom beserta tipe datanya pada masing-masing tabel
3. Hubungan: relasi antar entitas beserta tipe hubungannya

Berikut adalah contoh database schema atau ERD untuk data `chinook.db`:

<img src="assets/SQL/chinookschema2.png" width="75%">

### Apa arti simbol di garis penghubung antar tabel pada database schema?

Simbol pada garis penghubung antara Primary Key (PK) dan Foreign Key (FK) disebut juga sebagai **kardinalitas**. Ada banyak notasi yang digunakan untuk menyimbolkan sebuah kardinalitas, misalnya nformation Engineering Style, Chen Style, Bachman Style, Martin Style, dan lain-lain. Berikut adalah notasi Crows Foot yang digunakan pada ERD `chinook.db`:

<img src="assets/SQL/crows-foot.png" width="30%">

Sehingga, berikut adalah cara membaca kardinalitas pada database `chinook.db`:
- Satu `tracks` dapat memiliki satu atau lebih `genres`
- Satu `tracks` hanya dapat memiliki satu `media_types`, `playlist_track`, dan terdaftar pada tepat satu `invoice_items`
- Satu `playlists` hanya dapat terdaftar pada satu `playlist_track`
- Satu `albums` dapat memiliki satu atau lebih `tracks`
- Satu `albums` hanya dapat terdaftar pada satu `artists`
- Satu `invoice_items` hanya dapat terdaftar pada satu `invoices`
- Satu `invoices` hanya dapat dimiliki oleh satu `customers`
- `customers` dan `employees` memiliki hubungan yang opsional, karena bisa saja ada `employees` yang tidak berhubungan langsung dengan `customers` (misalkan divisi IT)
- `employees` memiliki hubungan yang opsional dengan dirinya sendiri, karena `employees` dengan jabatan paling tinggi (General Manager) tidak memerlukan `ReportsTo`

Tidak ada aturan baku dalam membuat kardinalitas, karena disesuaikan berdasarkan kasusnya. Penentuan kardinalitas masuk ke dalam proses database design yang merupakan tanggung jawab seorang database architect, di luar dari scope seorang data analyst atau data scientist.

### Bagaimana cara mengetahui list table dan kolom pada sqlite melalui query apabila database schema tidak diberikan?

Informasi detail mengenai database dapat diperoleh melalui `sqlite_master` seperti di bawah ini:

In [2]:
tables = pd.read_sql_query(
    '''
    SELECT name, sql
    FROM sqlite_master
    WHERE type = 'table' AND name NOT LIKE 'sqlite_%'
    ''', conn, index_col='name')
tables

,sql
name,
albums,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN..."
artists,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ..."
customers,"CREATE TABLE ""customers""\r\n(\r\n [Customer..."
employees,"CREATE TABLE ""employees""\r\n(\r\n [Employee..."
genres,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN..."
invoices,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId..."
invoice_items,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo..."
media_types,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT..."
playlists,"CREATE TABLE ""playlists""\r\n(\r\n [Playlist..."


Untuk melihat detail kolom pada masing-masing tabel, silahkan akses DataFrame `tables` sesuai nama tabelnya. Contohnya kita ingin melihat struktur kolom dari tabel `albums`:

In [3]:
print(tables.loc['albums', 'sql'])

CREATE TABLE "albums"
(
    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


SQL query tersebut disebut sebagai **Data Definition Language (DDL)**. Berikut adalah informasi yang ditampilkan:
- Nama kolom, contoh: `[AlbumId]`, `[Title]`, `[ArtistId]`
- Tipe data, contoh: `INTEGER`, `NVARCHAR(160)`
- Kolom unique sebagai key, contoh: `PRIMARY KEY`, `FOREIGN KEY`
- Pengaturan lainnya, contoh: `AUTOINCREMENT`, `NOT NULL`

### Apakah ada cara untuk auto-generate database schema dalam bentuk diagram?

Ada beberapa software yang dapat kita gunakan untuk membuat diagram secara otomatis dari database yang dimiliki, diantaranya adalah:
- [SchemaSpy](http://schemaspy.org/)
- [SQL Schema Explorer](https://timabell.github.io/schema-explorer/)
- [DbVisualizer](https://www.dbvis.com/)

## Database and SQL Concepts

### Apa itu API dan DBAPI?

API (Application Programming Interface) adalah suatu aplikasi sebagai **jembatan antara server dan client** agar kita bisa melakukan fungsi tertentu tanpa mengetahui bagaimana proses dibalik fungsi tersebut. Salah satu contoh API yang telah kita gunakan di kelas adalah `pandas-datareader` untuk mengakses data saham dari Yahoo! Finance. Berikut adalah ilutrasi dari analogi API:

<img src="assets/SQL/api.png" width="70%">

Sedangkan DBAPI adalah API yang spesifik digunakan untuk mengakses atau membangun koneksi dengan **database (DB)**. Kalau di Python sendiri, wujud DBAPI yaitu berupa package. Contoh DBAPI untuk SQLite adalah `sqlite3` sedangkan untuk MySQL adalah `pymysql`. Berikut ilustrasi DBAPI pada internal system dan API sebagai jembatan akses data untuk aplikasi pihak ketiga:

<img src="assets/SQL/dbapi.png" width="70%">

### Apakah menggunakan SQL kita hanya dapat melakukan fetching data (`SELECT`) atau bisa mengubah datanya juga?

Pada SQL terdapat banyak command yang dapat digunakan untuk berinteraksi dengan database yang dimiliki. Berdasarkan fungsinya, bahasa pada SQL query dapat dibagi menjadi empat:
1. Data Manipulation Language (DML): digunakan untuk mengoperasikan **data** pada tabel
> Query: `INSERT`, **`SELECT`**, `UPDATE`, `DELETE`
2. Data Definition Language (DDL): digunakan untuk mengoperasikan **struktur tabel**
> Query: `CREATE`, `ALTER`, `RENAME`, `DROP`
3. Data Control Language (DCL): digunakan untuk pengaturan **akses** database
> Query: `GRANT`, `REVOKE`
4. Transaction Processing Language (TCL): digunakan untuk **versioning** query
> Query: `ROLLBACK`, `COMMIT`

DML adalah query yang sering digunakan untuk Data Analyst maupun Data Scientist. Berikut adalah contohnya selain `SELECT`:

1. `INSERT`: menambahkan observasi baru pada tabel

Misalkan kita ingin menambahkan 3 `artists` baru, yaitu Artist1, Artist2, dan Artist3:

In [4]:
cursor = conn.cursor()
insert_data = [('Artist1', ),
               ('Artist2', ),
               ('Artist3', )]
cursor.executemany('INSERT INTO artists(Name) VALUES(?)', insert_data)
conn.commit()

In [5]:
pd.read_sql_query('SELECT * FROM artists', conn).tail()

,ArtistId,Name
273,274,Nash Ensemble
274,275,Philip Glass Ensemble
275,276,Artist1
276,277,Artist2
277,278,Artist3


2. `UPDATE`: mengubah nilai suatu observasi pada tabel

Misalkan kita ingin mengubah `Name` dari `artists` berdasarkan `ArtistId`:

In [6]:
cursor = conn.cursor()
edit_data = [('UpdatedArtist1', 276),
             ('UpdatedArtist2', 277),
             ('UpdatedArtist3', 278)]
cursor.executemany('UPDATE artists SET Name = ? WHERE ArtistId = ?', edit_data)
conn.commit()

In [7]:
pd.read_sql_query('SELECT * FROM artists', conn).tail()

,ArtistId,Name
273,274,Nash Ensemble
274,275,Philip Glass Ensemble
275,276,UpdatedArtist1
276,277,UpdatedArtist2
277,278,UpdatedArtist3


3. `DELETE`: menghapus observasi pada tabel

Misalkan kita ingin menghapus `artists` berdasarkan `ArtistId`:

In [8]:
cursor = conn.cursor()
delete_data = [(276,),
               (277,),
               (278,)]
cursor.executemany('DELETE FROM artists WHERE ArtistId = ?', delete_data)
conn.commit()

In [9]:
pd.read_sql_query('SELECT * FROM artists', conn).tail()

,ArtistId,Name
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble
274,275,Philip Glass Ensemble


Code berikut digunakan untuk me-reset kembali `AUTOINCREMENT` dari kolom `ArtistId` agar mulai seperti semula.

In [10]:
cursor = conn.cursor()
cursor.execute('UPDATE SQLITE_SEQUENCE SET SEQ=275 WHERE NAME="artists"')
conn.commit()

## SQL Query in Python

### Apakah itu `sqlite3`? Adakah konektor database selain `sqlite3`?

SQLite adalah sistem basis data yang menggunakan file dalam penyimpanannya (contohnya `chinook.db`) sehingga tidak memerlukan sebuah server, semua terjadi di sisi client. `sqlite3` adalah Database API (DBAPI) di Python untuk melakukan koneksi ke SQLite. [Documentation sqlite3](https://docs.python.org/3/library/sqlite3.html)

Terdapat konektor database lainnya tergantung sistem manajemen database (DBMS) yang digunakan, berikut adalah diantaranya yang umum digunakan:

- `pymysql` adalah DBAPI untuk MySQL, yaitu open-source relational DBMS yang sifatnya client-server dan umumnya digunakan untuk database sebuah website. [Documentation pymysql](https://pypi.org/project/PyMySQL/)

        import pymysql
        
        conn = pymysql.connect(
                host = HOST_NAME,
                port = PORT_NUMBER,
                user = USER_NAME,
                password = PASSWORD,
                db = DATABASE_NAME)

- `cx_Oracle` adalah DBAPI untuk Oracle Database, yaitu object-relational DBMS yang umumnya digunakan untuk online transaction processing dan data warehousing. [Documentation cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/index.html).

        import cx_Oracle
        
        # data source name from tnsnames.ora file
        dsn_tns = cx_Oracle.makedsn(
                    HOST_NAME,
                    PORT_NUMBER
                    service_name = SERVICE_NAME)
        
        # connection
        conn = cx_Oracle.connect(
                    user = USER_NAME,
                    password = PASSWORD,
                    dsn = dsn_tns)

- `psycopg2` adalah DBAPI untuk PostgreSQL, yaitu open-source object-relational DBMS. [Documentation psycopg](https://www.psycopg.org/docs/).

        import psycopg2
        
        conn = psycopg2.connect(
            host = HOST_NAME,
            port = PORT_NUMBER,
            user = USER_NAME,
            password = PASSWORD,
            database = DATABASE_NAME)

### Apakah ada alternatif menggunakan backslash `\` untuk meminimalisir kesalahan pengetikan query?

Terdapat resiko kesalahan pengetikan yang tinggi ketika menggunakan `\` sehingga diperlukan ketelitian yang lebih pada contoh query berikut:

In [11]:
pd.read_sql_query("SELECT * \
                   FROM artists \
                   LIMIT 5", conn)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


Alternatifnya adalah dengan menggunakan kutip sebanyak tiga kali `'''` atau `"""` di antara query sehingga kita bebas melakukan enter tanpa menggunakan `\`. 

In [12]:
pd.read_sql_query(
    '''
    SELECT *
    FROM artists
    LIMIT 5
    ''', conn)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


### Adakah ketentuan urutan pembacaan tabel saat melakukan operasi LEFT JOIN? Jika ya, sebaiknya tabel mana yang didahulukan?

Pada operasi LEFT JOIN, urutan pembacaan tabel dapat mempengaruhi hasil query apabila antar tabel memiliki hubungan yang opsional. Urutan pembacaan tabel disesuaikan dengan kebutuhan. Posisikan sebuah tabel sebagai left table apabila kita ingin mempertahankan seluruh baris pada tabel tersebut, mengizinkan adanya missing value pada data.

Contohnya pada tabel `employees` dengan `customers` hubungannya opsional karena tidak semua `employees` melayani `customers` secara langsung, hanya divisi Sales saja.

Case 1: Apabila kita ingin menampilkan semua data `employees` terlepas dari ada atau tidaknya `customers` yang dilayani, maka letakkan `employees` sebagai left table:

In [13]:
case1 = pd.read_sql_query(
    '''
    SELECT *
    FROM employees
    LEFT JOIN customers
    ON employees.EmployeeId = customers.SupportRepId
    ''', conn)

case1

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,...,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,...,None,None,None,None,None,None,None,None,None,NaN
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,...,None,None,None,None,None,None,None,None,None,NaN
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,...,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3.0
3,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,...,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3.0
4,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,...,Riotur,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,+55 (21) 2271-7000,+55 (21) 2271-7070,roberto.almeida@riotur.gov.br,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,...,None,110 Raeburn Pl,Edinburgh,None,United Kingdom,EH4 1HH,+44 0131 315 3300,None,steve.murray@yahoo.uk,5.0
60,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,...,None,"Calle Lira, 198",Santiago,None,Chile,None,+56 (0)2 635 4444,None,luisrojas@yahoo.cl,5.0
61,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,...,None,None,None,None,None,None,None,None,None,NaN
62,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,...,None,None,None,None,None,None,None,None,None,NaN


Case 2: Sebaliknya, apabila kita hanya ingin menampilkan semua data `customers` yang pernah dilayani oleh seorang `employees`, maka letakkan `customers` sebagai left table:

In [14]:
case2 = pd.read_sql_query(
    '''
    SELECT *
    FROM customers
    LEFT JOIN employees
    ON employees.EmployeeId = customers.SupportRepId
    ''', conn)

case2

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,...,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,...,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,...,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,...,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,...,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,...,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,...,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,...,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
7,8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,...,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
8,9,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,...,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
9,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,...,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com


Apabila diperhatikan, kedua tabel memiliki jumlah baris yang berbeda yaitu 5, menandakan ada 5 `employees` yang tidak pernah melayani `customers` sama sekali.

In [15]:
print(f"employees LEFT JOIN customers: {case1.shape}")
print(f"customers LEFT JOIN employees: {case2.shape}")

employees LEFT JOIN customers: (64, 28)
customers LEFT JOIN employees: (59, 28)


### Adakah operasi JOIN tabel pada `pandas`? Jika ada, apa perbedaannya dengan di SQL?

Misal kita ingin mereplikasi hasil query tabel berikut:

In [16]:
pd.read_sql_query(
    '''
    SELECT albums.*, Name
    FROM albums
    LEFT JOIN artists
    ON albums.ArtistId = artists.ArtistId
    ''', conn)

,AlbumId,Title,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith
...,...,...,...,...
342,343,Respighi:Pines of Rome,226,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,272,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,274,Nash Ensemble


Misalkan kita memiliki dua tabel yang terpisah, yaitu `albums` dan `artists`. Dengan menggunakan method `.join()` kita dapat melakukan operasi yang sama persis dengan SQL JOIN.

In [17]:
albums = pd.read_sql_query("SELECT * FROM albums", conn)
artists = pd.read_sql_query("SELECT * FROM artists", conn)

albums.join(artists.set_index('ArtistId'), on='ArtistId', how='left')

,AlbumId,Title,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith
...,...,...,...,...
342,343,Respighi:Pines of Rome,226,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,272,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,274,Nash Ensemble


Walaupun hasilnya sama, terdapat perbedaan penggunaan SQL JOIN dan `pandas` method `.join()`:
- Operasi JOIN pada SQL dilakukan di sisi server kemudian dikembalikan hasil gabungannya
- Method `.join()` `pandas` dilakukan di sisi client dan masing-masing tabel masih tersimpan pada memory

Sehingga, apabila kita memiliki data dengan volume yang besar (misal milyaran baris), maka penggunaan operasi JOIN dengan SQL akan jauh lebih cepat dibandingkan `.join()` pada `pandas`.

Silahkan simak kembali materi Under dan Over fetching pada course SQL.

### Bagaimana cara konversi data SQLite menjadi flat files (misalnya Excel)?

Kasus: kita ingin mengubah data `chinook.db` menjadi file Excel. Dalam kasus ini masih memungkinkan dikarenakan ukuran database cukup kecil. Apabila volume data sudah sangat besar, tidak sarankan menggunakan flat files. Berikut adalah langkahnya:

Pertama, kita peroleh daftar nama tabel yang tersimpan pada database `chinook.db` melalui `sqlite_master`

In [18]:
table_list = pd.read_sql_query(
    '''
    SELECT name
    FROM sqlite_master
    WHERE type = 'table' AND name NOT LIKE 'sqlite_%'
    ''', conn)['name'].values
table_list

array(['albums', 'artists', 'customers', 'employees', 'genres',
       'invoices', 'invoice_items', 'media_types', 'playlists',
       'playlist_track', 'tracks'], dtype=object)

Kemudian secara iteratif `pandas` membaca tabel dan menyimpannya ke file Excel dengan nama sheet yang berbeda.

In [19]:
writer = pd.ExcelWriter('data_input/chinook.xlsx', engine='xlsxwriter')   
for table in table_list:
    query = f"SELECT * FROM {table}"
    df = pd.read_sql_query(query, conn)
    df.to_excel(writer, sheet_name=table, index=False)   
writer.save()

Berikut adalah contoh output `chinook.xlsx` yang dihasilkan dari code di atas:

<img src="assets/SQL/chinook-excel.png" width="75%">

### Sebaliknya, bagaimana cara konversi data flat files menjadi SQLite?

Kasus: kita ingin mengubah kembali data `chinook.xlsx` menjadi `chinook_new.db` dimana 1 sheet Excel akan menjadi 1 tabel pada SQLite. Berikut adalah langkahnya:

1. Siapkan object `connection` dan `cursor` untuk database yang baru, yaitu `chinook_new.db`.

In [20]:
conn_new = sqlite3.connect("data_input/chinook_new.db")
cursor = conn_new.cursor()

2. Siapkan juga Data Definition Language berupa perintah `CREATE TABLE` yang mendefinisikan masing-masing tabel berisi kolom beserta tipe datanya, dan hubungan antar tabel. Tahap ini merupakan ranah seorang database architect, anggap sudah disediakan melalui `chinook_DDL.txt`.

In [21]:
f = open("data_input/chinook_DDL.txt", 'r')
ddl_query_list = f.read().split(';')
f.close()

print(ddl_query_list[0]) # contoh DDL untuk tabel albums

CREATE TABLE IF NOT EXISTS "albums"
(
    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


3. Query yang tersimpan pada `ddl_query_list` dieksekusi satu per satu dengan `cursor` agar terbentuk struktur tabel yang sesuai.

In [22]:
for query in ddl_query_list:
    cursor.execute(query)

Melalui informasi database yang tersimpan pada `sqlite_master`, kita dapat memastikan bahwa semua tabel sudah terbentuk dengan baik. Sampai pada tahap ini, tabel pada `chinook_new.db` masih kosong, belum ada datanya.

In [23]:
pd.read_sql_query("SELECT * FROM sqlite_master", conn_new)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,albums,albums,2,"CREATE TABLE ""albums"" (\n""AlbumId"" INTEGER,\n ..."
2,table,artists,artists,4,"CREATE TABLE ""artists"" (\n""ArtistId"" INTEGER,\..."
3,table,customers,customers,5,"CREATE TABLE ""customers"" (\n""CustomerId"" INTEG..."
4,table,employees,employees,6,"CREATE TABLE ""employees"" (\n""EmployeeId"" INTEG..."
5,table,genres,genres,7,"CREATE TABLE ""genres"" (\n""GenreId"" INTEGER,\n ..."
6,table,invoices,invoices,8,"CREATE TABLE ""invoices"" (\n""InvoiceId"" INTEGER..."
7,table,invoice_items,invoice_items,9,"CREATE TABLE ""invoice_items"" (\n""InvoiceLineId..."
8,table,media_types,media_types,10,"CREATE TABLE ""media_types"" (\n""MediaTypeId"" IN..."
9,table,playlists,playlists,11,"CREATE TABLE ""playlists"" (\n""PlaylistId"" INTEG..."


In [24]:
# belum ada data
pd.read_sql_query("SELECT * FROM artists", conn_new)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble


4. Pada tahap akhir, kita "populate" database `chinook_new.db` dari data `chinook.xlsx` dengan menggunakan method `.to_sql()`. Pastikan kedua hal berikut:
    - Nama sheet pada Excel harus sesuai dengan nama tabel pada SQLite
    - Nama kolom pada setiap sheet Excel harus sesuai dengan perintah DDL yang telah didefinisikan

In [25]:
sheet_df_dict = pd.read_excel('data_input/chinook.xlsx', sheet_name=None)

for sheet_name, df in sheet_df_dict.items():
    df.to_sql(sheet_name, conn_new, if_exists='replace', index=False)

Kita pastikan bahwa data sudah dimasukkan ke dalam tabel dengan baik:

In [26]:
pd.read_sql_query("SELECT * FROM artists", conn_new)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble


In [27]:
# menutup semua koneksi database
conn.close()
conn_new.close()